# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

In [3]:
%%classpath add mvn
org.clustering4ever clustering4ever_2.11 0.9.6

In [4]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.2.1
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);

In [6]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Simple Application")
                        .master("local[4]")
                        .config("spark.ui.enabled", "false")
                        .getOrCreate()
val sc = spark.sparkContext

org.apache.spark.SparkContext@681c52ca

In [7]:
import scala.collection.mutable
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.math.distances.ContinuousDistance
import org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityScalar
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.clusterizables.EasyClusterizable
import org.apache.spark.storage.StorageLevel
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.hashing.LDLSH
import org.clustering4ever.spark.clustering.GradientAscent
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal

import scala.collection.mutable
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.math.distances.ContinuousDistance
import org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityScalar
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.clusterizables.EasyClusterizable
import org.apache.spark.storage.StorageLevel
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.hashing.LDLSH
import org.clustering4ever.spark.clustering.GradientAscent
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal


## Download dataset Aggregation

In [8]:
%%bash
wget -P /tmp/ http://www.clustering4ever.org/Datasets/DS1/DS1.csv
wget -P /tmp/ http://www.clustering4ever.org/Datasets/DS1/DS1-labels.csv
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2019-08-23 11:37:40--  http://www.clustering4ever.org/Datasets/DS1/DS1.csv
Résolution de www.clustering4ever.org (www.clustering4ever.org)… 62.210.16.62
Connexion à www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 73786 (72K) [text/csv]
Sauvegarde en : « /tmp/DS1.csv.1 »

     0K .......... .......... .......... .......... .......... 69%  744K 0s
    50K .......... .......... ..                              100%  662K=0,1s

2019-08-23 11:37:40 (717 KB/s) — « /tmp/DS1.csv.1 » sauvegardé [73786/73786]

--2019-08-23 11:37:40--  http://www.clustering4ever.org/Datasets/DS1/DS1-labels.csv
Résolution de www.clustering4ever.org (www.clustering4ever.org)… 62.210.16.62
Connexion à www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 19315 (19K) [text/csv]
Sauvegarde en : « /tmp/DS1-labels.csv.1 »

     0K .....

In [11]:
val path = "/tmp/aggregation.csv"
val data = scala.io.Source.fromFile(path).getLines.map( x => Array(x.split(",").map(_.toDouble):_*) ).zipWithIndex
  .map{ case (v, id) => EasyClusterizable(id.toLong, ScalarVector(v)) }.toSeq
val rdd = sc.parallelize(data)
val labelsPath = "/tmp/labels"
val labels = scala.io.Source.fromFile(labelsPath).getLines.toSeq.map(_.toInt)

[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

## Gradient Ascent parameters

In [12]:
val k = 80 // Main Gradient Ascent parameter, the k of knn gradient ascent
val k2 = 1 // Optional parameter which determine if studied point has to switch to its knn majority vote neighbor bucket
val epsilon = 0.01 // Stopping criteria for Gradient Ascent, every point have to moove less than epsilon in order to stop iteration
val maxNumberIter = 20 
val bucketLayers = 1 // Neighbor bucket layer take into account during Gradient Ascent, at least 1 is recommended for decent k values, the bigger it is the slower it become
val w = 1D // Don't observe any change by changing this value, let it to whatever you want
val bucketNumber = 8 // Approximatively DS_Size / 1000 for decent performance, the large buckets are the slower it becomes
val propConvStopIter = 1D // Supplementary stopping criteria, stop if propConvStopIter % of data have converged
val memoryExpensive = true // Choose for fast or slow computation at some memory cost (building of similarity matrix per bucket per iter or on the fly (more calculus))
val persistanceLVL: StorageLevel = StorageLevel.MEMORY_ONLY // Persistance Spark level 
val dim = 2 // Dim of your dataset
val lsh = LDLSH(dim, w)

LDLSH(2,1.0)

## Epsilon proximity parameters

In [13]:
// Main parameter epsilon can be approximate with average distance knn => "knn:88" or be expressed directly as a double "eps:3.67"
val epsilonEpsProx = "knn:40"
// val epsilonEpsProx = "eps:2.56"
val classicalMetric = Euclidean(false)
val bucketNumber = 8 // Number of buckets in LSH, Approximatively DS_Size / 1000 for decent performance, the large buckets are the slower it becomes
val bucketLayers = 1 // Neighbor bucket layer take into account during Gradient Ascent, at least 1 is recommended for decent k values, the bigger it is the slower it become
val methodChoice = "bydot:1" // Fusionning cluster methods, at least X dot have to satisfy fusionning condition with "bydot:X", X=1 is greatly recommended
val divisionFactor = 1 // Experimental, let it to 1, it is supposed to speedup process on large datasets with bigger values but doesn't affect results
val cmin = 0 // Fusion clusters that have less than X elements with their closest cluster, it is in O(c²) with c number of clusters.

0

In [14]:
// Theses 3 params have to be here else there is some bugs..., it works more properly on full programm versions
val idOriginalVector = 8
val idToStoreGAVector = 0
val vm = VMapping

val t0 = System.currentTimeMillis

val ascended = GradientAscent.train(rdd, k, epsilon, maxNumberIter, bucketNumber, bucketLayers, k2, propConvStopIter, memoryExpensive,
  persistanceLVL, idToStoreGAVector, true)


val updatedData = ascended.map( cz => cz.addAlternativeVector(idOriginalVector, cz.v).switchForExistingVectorization(idToStoreGAVector, vm) )

val model = EpsilonProximityScalar(
  epsilonEpsProx,
  methodChoice,
  classicalMetric,
  bucketNumber,
  lsh,
  cmin,
  bucketLayers,
  StorageLevel.MEMORY_ONLY,
  divisionFactor
).fit(updatedData)

val clusterizedRDD = model.obtainInputDataClustering(updatedData)

val predLabels = clusterizedRDD.map(_.clusterIDs.head).collect

val t1 = System.currentTimeMillis

val trueLabels = scala.io.Source.fromFile(labelsPath).getLines.map(_.toInt).toSeq.toArray
val targetAndPred = trueLabels.zip(predLabels)

(t1 - t0) / 1000D

<console>: 134